In [15]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os


In [14]:
# check folder status
folderLst = ['mdl', 'tab']
for folder in folderLst:
    if not os.path.exists('../' + folder):
        os.makedirs('../' + folder)

NameError: name 'os' is not defined

In [2]:
# load data without review; parse date
dat = pd.read_csv('../dat/yelp_academic_dataset_review_nr.csv', parse_dates=['date'])

In [3]:
# filter based on item counts
ITEM_THRES = 10

item_count = dat.business_id.value_counts()
print 'number of items: %d' % len(item_count)

dat = dat.set_index('business_id', drop=False)
dat['item_count'] = item_count
dat = dat.loc[dat['item_count'] >= ITEM_THRES]

number of items: 144072


In [4]:
# filter based on user counts
USER_THRES = 5

user_count = dat.user_id.value_counts()
print 'number of users: %d' % len(user_count)

dat = dat.set_index('user_id', drop=False)
dat['user_count'] = user_count
dat = dat.loc[dat['user_count'] >= USER_THRES]

number of users: 963325


In [5]:
# consider unique items and users
uni_item = dat.business_id.unique()
NI = len(uni_item)
uni_user = dat.user_id.unique()
NU = len(uni_user)
print 'number of items: %d' % NI
print 'number of users: %d' % NU

number of items: 67308
number of users: 163940


In [12]:
# write lists of users
with open('../tab/userList', 'w') as f:
    f.write(str(len(uni_user)) + '\n')
    f.write('\n'.join(uni_user))

# write lists of items
with open('../tab/itemList', 'w') as f:
    f.write(str(len(uni_item)) + '\n')
    f.write('\n'.join(uni_item))

In [6]:
# create mapping for item
item_map = dict()
for idx, item_id in enumerate(uni_item):
    item_map[item_id] = idx
    
# create mapping for user
user_map = dict()
for idx, user_id in enumerate(uni_user):
    user_map[user_id] = idx

In [7]:
# create dataset in tuple
tn = np.array([(user_map[u], item_map[i], r) \
               for u, i, r, t in dat.loc[:,['user_id', 'business_id', 'stars', 'date']].values \
               if t.year <= 2015])
tt = np.array([(user_map[u], item_map[i], r) \
               for u, i, r, t in dat.loc[:,['user_id', 'business_id', 'stars', 'date']].values \
               if t.year == 2016])

In [13]:
# write tn as tuple
with open('../tab/tnList', 'w') as f:
    f.write(str(tn.shape[0]) + '\n')
    f.write('\n'.join([','.join(map(str,e)) for e in tn]))
    
# write tn as tuple
with open('../tab/ttList', 'w') as f:
    f.write(str(tt.shape[0]) + '\n')
    f.write('\n'.join([','.join(map(str,e)) for e in tt]))

In [8]:
print tn.shape
print tt.shape

(1934990, 3)
(534135, 3)
